In [2]:
import pathlib
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt


import SessionTools.two_photon as st2p


%load_ext autoreload
%autoreload 2 

%matplotlib inline

In [3]:
session_info = {'date': "12_04_2025",
                'genotype_dir': "R56H10_R20A02_split_SynaptopHluorin_CsChrimsontdT",
                'fly': 'fly1',
                'session': 'baseline_opto_response-002',
                'full_genotype': '',
                'ecl date': '',
                'notes': ''}

bruker_folder = pathlib.PurePath('/media/mplitt/SSD_storage/2PData')

pp = st2p.preprocessing.EBImagingSession(session_info = session_info, 
                                         bruker_base_dir = bruker_folder,
                                        )


In [ ]:
pp.extract_metadata()
pp.bruker_to_h5(overwrite=False,first_chan=1)
data_corr, shifts, error, diffphase= pp.motion_correct(pp.data)


In [ ]:
nap = pp.open_napari()

In [ ]:
n_rois = 16
nap.make_phase_masks(n_rois=n_rois)
pp.get_layers(nap)
pp.extract_timeseries(max_proj=False)

In [ ]:
y_or = pp.calculate_zscored_F('outer_ring', exp_detrend=True,zscore=False)

In [ ]:
pp.save()

In [ ]:
fig, ax = plt.subplots(figsize = [ 15,3])
ax.plot(y_or.ravel())

In [4]:
def get_stims(pp):

    frame_times = np.array(pp.metadata['frame_times']).ravel()
    dt = (frame_times[1:]-frame_times[:-1]).mean()

    stim_times = [pp.metadata['mark_points']['initial_delay']]
    for i in range(pp.metadata['mark_points']['repetitions']-1):
        stim_times.append(pp.metadata['mark_points']['inter_point_delay']+pp.metadata['mark_points']['duration'])
    stim_times = np.array(stim_times)/1000
    stim_times = np.cumsum(stim_times)
    
    
    stim_inds = []
    for time in stim_times:
        stim_inds.append( np.argwhere((frame_times-time)>0)[0][0])
    stim_inds = np.array(stim_inds)
    
    stim_resp = np.zeros((len(stim_inds), 80))
    for i, ind in enumerate(stim_inds):
        print(i)
        stim_resp[i,:] = y_or[ind-10:ind+70]
    stim_resp -= np.nanmean(stim_resp[:,:10], axis=-1, keepdims=True)
    return frames_times, stim_inds, stim_resp

In [5]:
frame_times, stim_inds, stim_resp = get_stims(pp)

TypeError: 'NoneType' object is not subscriptable

In [ ]:
fig, ax= plt.subplots()
mu = np.mean(stim_resp[:,:],axis=0)
sem = sp.stats.sem(stim_resp,axis=0, nan_policy='omit')

x = np.arange(-10*dt, 69*dt+1E-3, dt) 
ax.plot(x, mu, color='blue')
ax.fill_between(x, mu-sem, mu+sem, color='blue', alpha=.3)
ax.set_xlabel('Time (s)')
ax.set_ylabel('dF/F')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.plot([0,0],[-.15,.2], 'k--',alpha=.4)
ax.set_title('Synapto-pHluorin-Vamp2 (50 microM Picrotoxin, 25 microM curare)')
